In [142]:
import pyomo.environ as pyo

from scipy.spatial.distance import pdist, squareform

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import time
from itertools import cycle

#from pyomo.contrib.appsi.solvers import Highs
# import gurobipy
import pandas as pd

In [143]:
data = pd.read_csv('data.csv',header = None)
data = data.to_numpy().flatten()
data.shape

(10400,)

In [144]:
data = data[~np.isnan(data)]
data = data.reshape((100,100))
data

array([[52., 89., 40., ..., 29., 46., 69.],
       [20., 17., 80., ..., 76., 88., 92.],
       [17., 34., 50., ..., 76., 39., 31.],
       ...,
       [10., 90., 89., ..., 23., 57., 79.],
       [45., 50., 73., ..., 11., 44., 29.],
       [60., 25., 73., ..., 64., 76.,  4.]])

In [145]:
A = 100
R = 100

In [147]:
data[50,5]

51.0

In [148]:
model = pyo.ConcreteModel()

In [149]:
model.I = pyo.Set(initialize = range(A)) # set of jobs to be done
model.J = pyo.Set(initialize = range(R)) # set of workers

## Variables

In [150]:
model.X = pyo.Var(model.I, model.J ,within=pyo.Binary)

## Constraints

In [151]:
def one_job_per_worker(model,j):
    return sum(model.X[i,j] for i in model.I) ==1 
model.one_job_per_worker = pyo.Constraint(model.J, rule=one_job_per_worker)

def one_worker_per_job(model,i):
    return sum(model.X[i,j] for j in model.J) ==1 
model.one_worker_per_job = pyo.Constraint(model.I, rule=one_worker_per_job)

In [152]:
model.obj = pyo.Objective(
    expr = sum(model.X[i,j] * data[i,j]  
    for i in model.I
    for j in model.J),
    sense = pyo.minimize,
)

In [ ]:
TIME_LIMIT = 60
THREADS = 4

SOVLER_ENGINE = 'appsi_highs'
#solvers glpk  appsi_highs cplex gurobi gurobi_persistent

solver = pyo.SolverFactory(SOVLER_ENGINE)

if SOVLER_ENGINE == 'cbc':
        solver.options['seconds'] = TIME_LIMIT
elif SOVLER_ENGINE == 'glpk':
        solver.options['tmlim'] = TIME_LIMIT
elif SOVLER_ENGINE == 'appsi_highs':
        solver.options['time_limit'] = TIME_LIMIT
        #solver.options['parallel'] = True
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'cplex':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'gurobi':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'gurobi_persistent':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
        solver.set_instance(model) # remove model from  solver.solve(tee=True)
        solver.set_gurobi_param("PoolSolutions", 500)
        solver.set_gurobi_param("PoolSearchMode", 0)
# sol = solver.solve(tee= True)
sol = solver.solve(model, tee= True) #, warmstart=True , logfile= 'log.txt'

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
200 rows, 10000 cols, 20000 nonzeros  0s
200 rows, 10000 cols, 20000 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   200 rows
   10000 cols (10000 binary, 0 integer, 0 implied int., 0 continuous)
   20000 nonzeros
MIP-Timing:        0.11 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSo

In [154]:
print(sol.solver.status)
print(sol.solver.termination_condition)
print (model.obj())

ok
optimal
305.0


In [155]:
node = 1
worker_list = []
jobs_list = []
for i in model.I:
    for j in model.J:
        if np.isclose(model.X[i, j].value, 1, atol=1e-1):
                worker_list.append(i)
                if j in jobs_list:
                     print('ERROR')
                else:
                     jobs_list.append(j)
                print ('Job:',i+1, "assigned to worker:", j+1, 'cost:' ,data[i,j])


Job: 1 assigned to worker: 78 cost: 6.0
Job: 2 assigned to worker: 7 cost: 4.0
Job: 3 assigned to worker: 30 cost: 2.0
Job: 4 assigned to worker: 40 cost: 2.0
Job: 5 assigned to worker: 76 cost: 2.0
Job: 6 assigned to worker: 18 cost: 4.0
Job: 7 assigned to worker: 85 cost: 3.0
Job: 8 assigned to worker: 32 cost: 5.0
Job: 9 assigned to worker: 10 cost: 2.0
Job: 10 assigned to worker: 28 cost: 5.0
Job: 11 assigned to worker: 58 cost: 4.0
Job: 12 assigned to worker: 73 cost: 4.0
Job: 13 assigned to worker: 14 cost: 2.0
Job: 14 assigned to worker: 1 cost: 3.0
Job: 15 assigned to worker: 71 cost: 2.0
Job: 16 assigned to worker: 67 cost: 4.0
Job: 17 assigned to worker: 24 cost: 2.0
Job: 18 assigned to worker: 86 cost: 5.0
Job: 19 assigned to worker: 5 cost: 3.0
Job: 20 assigned to worker: 95 cost: 2.0
Job: 21 assigned to worker: 96 cost: 4.0
Job: 22 assigned to worker: 65 cost: 2.0
Job: 23 assigned to worker: 19 cost: 4.0
Job: 24 assigned to worker: 94 cost: 3.0
Job: 25 assigned to worker: 